In [1]:
from elasticsearch import Elasticsearch, helpers
import os
import re
import json
import logging

# Parse files and convert to JSON format accepted by ES

In [2]:
def parse(u):

    buffer = ""
    docids = []
    doctexts = []
    
    try:
        for filename in os.listdir(u):
            with open(os.path.join(path, filename)) as f:
                buffer = ""
                for line in f:
                    if "</DOC>" in line:
                        docid = re.findall(r"<DOCNO> (.*?) </DOCNO>", buffer)
                        contents = re.findall(r"<TEXT> (.*?) </TEXT>", buffer, re.VERBOSE|re.MULTILINE|re.DOTALL)
                        contents[0].replace('<TEXT>',' ')
                        contents[0].replace('</TEXT>',' ')
                        docids.append(docid)
                        doctexts.append(contents)
                        buffer = ""
                    line = line.replace('\n',' ')
                    buffer = buffer + line

       
    except Exception as ex:
        print('Exception while parsing')
        print(str(ex))
    finally:
        # return json.dumps(rec)
        return json.dumps([{'_id': docid, 'text': contents} for docid, contents in zip(docids, doctexts)])
    

In [3]:
 path = "E:\\CS 6200\\AP_DATA\\ap89_collection"

In [4]:
json_data = parse(path)

Exception while parsing
list index out of range


In [5]:
item_dict = json.loads(json_data)
print(len(item_dict))

84678


In [6]:
def connect_elasticsearch():
    _es = None
    _es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
    if _es.ping():
        print('Yay Connected')
    else:
        print('Awww it could not connect!')
    return _es
if __name__ == '__main__':
  logging.basicConfig(level=logging.ERROR)

In [7]:
es = connect_elasticsearch()

Yay Connected


In [8]:
def create_index(es_object, index_name='assignment1'):
    created = False
    # index settings
    settings = {
       "settings" : {
        "number_of_shards": 1,
        "number_of_replicas": 1,
        "analysis": {
            "filter": {
                "english_stop": {
                    "type": "stop",
                    "stopwords_path": "my_stoplist.txt"
                },
                "my_stemmer": {
                    "type": "stemmer",
                    "name": "english"
                }
            },
            "analyzer": {
                "stopped_and_stemmed": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "english_stop",
                        "my_stemmer"
                    ]
                }
            }
      }
    },
    "mappings": {
        "properties": {
            "text": {
                "type": "text",
                "fielddata": True,
                "analyzer": "stopped_and_stemmed",
                "index_options": "positions"
            }
        }
    }
    }

    try:
        if not es_object.indices.exists(index_name):
            # Ignore 400 means to ignore "Index Already Exist" error.
            es_object.indices.create(index=index_name, ignore=400, body=settings)
            print('Created Index')
        created = True
    except Exception as ex:
        print(str(ex))
    finally:
        return created

In [9]:
create_index(es,'assignment1')

Created Index


True

In [10]:
for item in item_dict:
    if (len(item['_id'])==1):
        item['_id']=item['_id'][0]
    if (len(item['text'])==1):
        item['text']=item['text'][0]

In [11]:
def insertElasticsearch(esobj, bulkItems, doc_type="_doc"):
        docs = []
        for item in bulkItems:
            rec = {'text': item["text"]}
            doc = {
                '_id': item["_id"],
                '_index': 'assignment1',
                '_type': doc_type,
                '_source': json.dumps(rec)
            }
            docs.append(doc)

        res = helpers.bulk(esobj, actions=docs)
        return res 


In [12]:
insertElasticsearch(es, item_dict)

(84678, [])